In [ ]:
import os
import numpy as np
import cv2
import time
import rospy
import message_filters
from cv_bridge import CvBridge, CvBridgeError
from sensor_msgs.msg import Image


class SaveImage(object):

    def __init__(self, vis=False):
        self.rsidlist = ['rs15', 'rs16', 'rs17', 'rs18']
        self.save_dir = os.path.join('/home/ai-cap02/workspace/data',
                                     'apriltags')
        self.vis = vis
        self.rs_num = len(self.rsidlist)
        self.sequence = 0
        rospy.init_node('save_images', anonymous=True)

    def callback(self, color, depth, rsid):
        header = color.header
        secs = header.stamp.secs
        nsecs = header.stamp.nsecs
        color = CvBridge().imgmsg_to_cv2(color, desired_encoding=color.encoding)
        depth = CvBridge().imgmsg_to_cv2(depth, desired_encoding=depth.encoding)
        
        path_color = os.path.join(self.save_dir, rsid, 'color')
        path_depth = os.path.join(self.save_dir, rsid, 'depth')
        if not os.path.exists(path_color):
            os.makedirs(path_color)
        if not os.path.exists(path_depth):
            os.makedirs(path_depth)

        file_color = os.path.join(path_color, '%s.%s.jpg' %(secs, nsecs))
        file_depth = os.path.join(path_color, '%s.%s.png' %(secs, nsecs))

        cv2.imwrite(file_color, color)
        cv2.imwrite(file_depth, depth)
        print('[Info]: %s saved!' % file_color)
        
    def cb_vis(self, color, depth, rsid):
        color = CvBridge().imgmsg_to_cv2(color, desired_encoding=color.encoding)
        depth = CvBridge().imgmsg_to_cv2(depth, desired_encoding=depth.encoding)
        
        path = os.path.join(self.save_dir, 'test')
        if not os.path.exists(path):
            os.makedirs(path)
        cv2.imshow('%s' % rsid, color)
        k = cv2.waitKey(1)
        if k & 0xFF == ord('q') or k == 27:
            cv2.destroyAllWindows()
            rospy.signal_shutdown('Exit.')
            print('[End]: Exitting...')
            sys.exit(0)
            
        if k & 0xFF == ord('s') or k == ord(' '):
            file_color = os.path.join(path, '%s_%d.jpg' %(rsid, self.sequence))
            file_depth = os.path.join(path, '%s_%d.png' %(rsid, self.sequence))
            cv2.imwrite(file_color, color)
            cv2.imwrite(file_depth, depth)
            self.sequence += 1
            print('[Info]: %s saved!' % os.path.basename(file_color))
    
    def cb_multi(self, *args):
        
        color_list = []
        depth_list = []
        
        for i in range(self.rs_num):
            color_msg = args[i*2]
            color = CvBridge().imgmsg_to_cv2(color_msg, 
                        desired_encoding=color_msg.encoding)
            color_list.append(color)
            
            depth_msg = args[i*2 + 1]
            depth = CvBridge().imgmsg_to_cv2(depth_msg, 
                        desired_encoding=depth_msg.encoding)
            depth_list.append(depth)
            
        rsid_list = args[-1]
        
        
        # For 4 cameras.
        colors_toshow = [cv2.resize(each, (424, 240)) for each in color_list]
        h_imgs_1 = np.hstack([colors_toshow[0], colors_toshow[1]])
        h_imgs_2 = np.hstack([colors_toshow[2], colors_toshow[3]])
        all_imgs = np.vstack([h_imgs_1, h_imgs_2])
        
        path = os.path.join(self.save_dir, 'test')
        if not os.path.exists(path):
            os.makedirs(path)
        
        cv2.imshow('All RS', all_imgs)
        k = cv2.waitKey(50)
        
        if k == ord('q') or k == 27:
            cv2.destroyAllWindows()
            rospy.signal_shutdown('Exit.')
            print('[End]: Exitting...')
            sys.exit(0)
            
        if k == ord('s') or k == ord(' '):
            for i in range(self.rs_num):
                file_color = os.path.join(path, '%s_%d.jpg' %(rsid_list[i], self.sequence))
                file_depth = os.path.join(path, '%s_%d.png' %(rsid_list[i], self.sequence))
                cv2.imwrite(file_color, color_list[i])
                cv2.imwrite(file_depth, depth_list[i])
            
            self.sequence += 1
            print('[Info]: %s saved!' % os.path.basename(file_color))
        
            
    def run(self):
        if self.vis:
            cb = self.cb_multi
        else:
            cb = self.callback
            
        sublist = []
        for rsid in self.rsidlist:
            color_sub = message_filters.Subscriber('/' + rsid + '/color/image_rect_color', Image)
            depth_sub = message_filters.Subscriber('/' + rsid + '/aligned_depth_to_color/image_raw', Image)
            sublist.append(color_sub)
            sublist.append(depth_sub)
            
        ts = message_filters.ApproximateTimeSynchronizer(sublist, 99, 0.5)
        ts.registerCallback(cb, self.rsidlist)

        rospy.spin()



if __name__ == '__main__':
    S = SaveImage(vis=True)
    S.run()
